In [2]:
import json
path = '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/attributes.json'
with open(path, 'r') as f:
    data = json.load(f) 
print(data)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [3]:
path = '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/objects.json'
with open(path, 'r') as f:
    data = json.load(f) 
print(data[0])

{'image_id': 1, 'objects': [{'synsets': ['tree.n.01'], 'h': 557, 'object_id': 1058549, 'merged_object_ids': [], 'names': ['trees'], 'w': 799, 'y': 0, 'x': 0}, {'synsets': ['sidewalk.n.01'], 'h': 290, 'object_id': 1058534, 'merged_object_ids': [5046], 'names': ['sidewalk'], 'w': 722, 'y': 308, 'x': 78}, {'synsets': ['building.n.01'], 'h': 538, 'object_id': 1058508, 'merged_object_ids': [], 'names': ['building'], 'w': 222, 'y': 0, 'x': 1}, {'synsets': ['street.n.01'], 'h': 258, 'object_id': 1058539, 'merged_object_ids': [3798578], 'names': ['street'], 'w': 359, 'y': 283, 'x': 439}, {'synsets': ['wall.n.01'], 'h': 535, 'object_id': 1058543, 'merged_object_ids': [], 'names': ['wall'], 'w': 135, 'y': 1, 'x': 0}, {'synsets': ['tree.n.01'], 'h': 360, 'object_id': 1058545, 'merged_object_ids': [], 'names': ['tree'], 'w': 476, 'y': 0, 'x': 178}, {'synsets': ['shade.n.01'], 'h': 189, 'object_id': 5045, 'merged_object_ids': [], 'names': ['shade'], 'w': 274, 'y': 344, 'x': 116}, {'synsets': ['van.

In [6]:
path = '/home/rfsm2/rds/hpc-work/datasets/vg/raw_vg/scene_graphs.json'
with open(path, 'r') as f:
    data = json.load(f) 
print(data[0])

{'relationships': [{'synsets': ['along.r.01'], 'predicate': 'ON', 'relationship_id': 15927, 'object_id': 5046, 'subject_id': 5045}, {'synsets': ['wear.v.01'], 'predicate': 'wears', 'relationship_id': 15928, 'object_id': 5048, 'subject_id': 1058529}, {'synsets': ['have.v.01'], 'predicate': 'has', 'relationship_id': 15929, 'object_id': 5050, 'subject_id': 5049}, {'synsets': ['along.r.01'], 'predicate': 'ON', 'relationship_id': 15930, 'object_id': 1058508, 'subject_id': 1058507}, {'synsets': ['along.r.01'], 'predicate': 'ON', 'relationship_id': 15931, 'object_id': 1058534, 'subject_id': 5055}, {'synsets': ['have.v.01'], 'predicate': 'has', 'relationship_id': 15932, 'object_id': 1058511, 'subject_id': 1058529}, {'synsets': ['next.r.01'], 'predicate': 'next to', 'relationship_id': 15933, 'object_id': 1058539, 'subject_id': 1058534}, {'synsets': ['have.v.01'], 'predicate': 'has', 'relationship_id': 15934, 'object_id': 5060, 'subject_id': 1058515}, {'synsets': ['have.v.01'], 'predicate': 'has

In [7]:
print(data[0].keys())

dict_keys(['relationships', 'image_id', 'objects'])


In [16]:
data[0]['relationships']

[{'synsets': ['along.r.01'],
  'predicate': 'ON',
  'relationship_id': 15927,
  'object_id': 5046,
  'subject_id': 5045},
 {'synsets': ['wear.v.01'],
  'predicate': 'wears',
  'relationship_id': 15928,
  'object_id': 5048,
  'subject_id': 1058529},
 {'synsets': ['have.v.01'],
  'predicate': 'has',
  'relationship_id': 15929,
  'object_id': 5050,
  'subject_id': 5049},
 {'synsets': ['along.r.01'],
  'predicate': 'ON',
  'relationship_id': 15930,
  'object_id': 1058508,
  'subject_id': 1058507},
 {'synsets': ['along.r.01'],
  'predicate': 'ON',
  'relationship_id': 15931,
  'object_id': 1058534,
  'subject_id': 5055},
 {'synsets': ['have.v.01'],
  'predicate': 'has',
  'relationship_id': 15932,
  'object_id': 1058511,
  'subject_id': 1058529},
 {'synsets': ['next.r.01'],
  'predicate': 'next to',
  'relationship_id': 15933,
  'object_id': 1058539,
  'subject_id': 1058534},
 {'synsets': ['have.v.01'],
  'predicate': 'has',
  'relationship_id': 15934,
  'object_id': 5060,
  'subject_id': 1

In [12]:
data[0]['image_id']
data[0]['regions']

[{'relationships': [],
  'region_id': 1382,
  'width': 82,
  'synsets': [{'entity_idx_start': 4,
    'entity_idx_end': 9,
    'entity_name': 'clock',
    'synset_name': 'clock.n.01'},
   {'entity_idx_start': 22,
    'entity_idx_end': 28,
    'entity_name': 'colour',
    'synset_name': 'color.n.04'}],
  'height': 139,
  'image_id': 1,
  'objects': [{'name': 'clock',
    'h': 339,
    'object_id': 1058498,
    'synsets': ['clock.n.01'],
    'w': 79,
    'y': 91,
    'x': 421}],
  'phrase': 'the clock is green in colour',
  'y': 57,
  'x': 421},
 {'relationships': [{'synsets': ['along.r.01'],
    'predicate': 'ON',
    'relationship_id': 15927,
    'object_id': 5046,
    'subject_id': 5045}],
  'region_id': 1383,
  'width': 182,
  'synsets': [{'entity_idx_start': 0,
    'entity_idx_end': 5,
    'entity_name': 'shade',
    'synset_name': 'shade.n.01'},
   {'entity_idx_start': 19,
    'entity_idx_end': 25,
    'entity_name': 'street',
    'synset_name': 'street.n.01'}],
  'height': 109,
  '